In [1]:
from pathlib import Path
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


DATA_PATH = Path.cwd().parent / "data" if Path.cwd().name == "src" else Path.cwd() / "data"
DATA_PATH

E0000 00:00:1732298375.076176   77935 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732298375.079861   77935 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PosixPath('/workspaces/ds340-project/data')

In [2]:
from tensorflow.config import list_physical_devices
print("Num GPUs Available: ", len(list_physical_devices('GPU')))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()[1])

tf.config.experimental.set_memory_growth(tf.config.list_physical_devices()[1], True)

Num GPUs Available:  1
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5833228288
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9311826414884405339
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419



I0000 00:00:1732298410.107232   77935 gpu_device.cc:2022] Created device /device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
image_size = (224, 224)
batch_size = 64

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH / "base_input" / "train",
    validation_split=0.2,
    subset="training",
    seed=340,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH / "base_input" / "valid",
    validation_split=0.2,
    subset="validation",
    seed=340,
    image_size=image_size,
    batch_size=batch_size,
)

# test_ds = tf.keras.utils.image_dataset_from_directory(
#     DATA_PATH / "nobg_input" / "test",
#     validation_split=0.2,
#     subset="validation",
#     seed=340,
#     image_size=image_size,
#     batch_size=batch_size,
# )

Found 75736 files belonging to 101 classes.
Using 60589 files for training.


I0000 00:00:1732298434.274495   77935 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 15150 files belonging to 101 classes.
Using 3030 files for validation.


In [4]:
input_shape = (*image_size, 3)
num_classes = 101

In [5]:
def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=num_classes)
    return image, label

train_ds = train_ds.map(one_hot_encode)
val_ds = val_ds.map(one_hot_encode)
# test_ds = test_ds.map(one_hot_encode)

In [6]:
from tensorflow.keras.applications import EfficientNetB0
from keras.models import Model

base_model = EfficientNetB0(include_top=False, input_shape=input_shape, pooling='avg')

for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.layers[-1].output)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(101, activation='softmax')(x)
# output = layers.Dense(101, activation='softmax')(base_model.layers[-1].output)
model = Model(inputs=base_model.inputs, outputs=output)


In [8]:
FINETUNE = True
epochs = 20
checkpoint_filepath = Path().cwd() / "model_ckpts" / "base_model.keras"
lr = 0.001

if checkpoint_filepath.exists() and FINETUNE:
    model = keras.models.load_model(checkpoint_filepath, compile=False)
    lr = 0.0002
    
    for layer in model.layers:
        layer.trainable = True

    print(f"Model loaded from: {checkpoint_filepath}")

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=["accuracy"])

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=3
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
] 

model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)

Model loaded from: /workspaces/ds340-project/src/model_ckpts/base_model.keras
Epoch 1/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 0.4430 - loss: 2.4264
Epoch 1: val_accuracy improved from -inf to 0.76304, saving model to /workspaces/ds340-project/src/model_ckpts/base_model.keras
947/947 ━━━━━━━━━━━━━━━━━━━━ 265s 214ms/step - accuracy: 0.4431 - loss: 2.4259 - val_accuracy: 0.7630 - val_loss: 0.9119
Epoch 2/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.6355 - loss: 1.3731
Epoch 2: val_accuracy improved from 0.76304 to 0.77426, saving model to /workspaces/ds340-project/src/model_ckpts/base_model.keras
947/947 ━━━━━━━━━━━━━━━━━━━━ 158s 167ms/step - accuracy: 0.6355 - loss: 1.3731 - val_accuracy: 0.7743 - val_loss: 0.8046
Epoch 3/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.6897 - loss: 1.1427
Epoch 3: val_accuracy improved from 0.77426 to 0.80528, saving model to /workspaces/ds340-project/src/model_ckpts/base_model.keras
947/947 ━━━━━━━━━━━━━━━━━━